In [ ]:
# configs
storage_account_name = "[storage_account_name]"
tenant_id = "[tenant_id]"
client_id = "[client_id]"
client_secret = "[client_secret]"

# spark configuration options
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net","OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [ ]:
from pyspark.sql.functions import col
from datetime import datetime

dataverse_customer_table_name = '[dataverse_customer_table_name]'
bronze_container_name = 'bronze'
silver_container_name = 'silver'
partition_date = datetime.now().strftime("%Y-%m-%d")

raw_table_df = spark.read.option("header", "true").option("multiLine", "true").csv(f"abfss://{bronze_container_name}@{storage_account_name}.dfs.core.windows.net/{dataverse_customer_table_name}/{partition_date}")

raw_table_df = raw_table_df.withColumnRenamed("crc33_postcode", "PostCode")
raw_table_df = raw_table_df.withColumnRenamed("crc33_state", "State")
raw_table_df = raw_table_df.withColumnRenamed("crc33_lastname", "LastName")
raw_table_df = raw_table_df.withColumnRenamed("crc33_country", "Country")
raw_table_df = raw_table_df.withColumnRenamed("crc33_gender", "Gender")
raw_table_df = raw_table_df.withColumnRenamed("crc33_streetaddress", "StreetAddress")
raw_table_df = raw_table_df.withColumnRenamed("crc33_name", "Name")
raw_table_df = raw_table_df.withColumnRenamed("crc33_telephone", "Telephone")
raw_table_df = raw_table_df.withColumnRenamed("crc33_dateofbirth", "DateOfBirth")
raw_table_df = raw_table_df.withColumnRenamed("crc33_city", "City")
raw_table_df = raw_table_df.withColumnRenamed("crc33_firstname", "FirstName")
raw_table_df = raw_table_df.withColumnRenamed("crc33_email", "Email")
raw_table_df = raw_table_df.withColumnRenamed("crc33_cid", "cid")

silver_table_df = raw_table_df.select(col("cid"),col("postcode"), col("state"),col("lastname"),col("country"),col("gender"),col("streetaddress"),col("name"),col("telephone"),col("dateofbirth"),col("city"),col("firstname"),col("email"))

silver_table_df.write.mode('append').parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{dataverse_customer_table_name}")

In [ ]:
customer_spend_table_name = 'customer_spend'

customer_spend_df = spark.read.option("header", "true").csv(f"abfss://{bronze_container_name}@{storage_account_name}.dfs.core.windows.net/{customer_spend_table_name}")
customer_spend_df.write.mode('append').parquet(f"abfss://{silver_container_name}@{storage_account_name}.dfs.core.windows.net/{customer_spend_table_name}")